In [1]:
%matplotlib inline

import pandas as  pd
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
import seaborn as sns

from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection
import csv

# just for the sake of this blog post!
from warnings import filterwarnings
filterwarnings('ignore')

In [53]:
train_features = pd.read_csv("data/dengue_features_train.csv")
train_labels = pd.read_csv("data/dengue_labels_train.csv")

In [54]:
train_features

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
5,sj,1990,23,1990-06-04,NaN,0.174850,0.254314,0.181743,9.58,299.630000,...,26.49,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,...,38.60,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,...,30.00,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,...,37.51,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1
9,sj,1990,27,1990-07-02,NaN,0.121550,0.160683,0.202567,14.41,300.154286,...,28.40,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1


In [57]:
'''
AFTER EXPLANATORY ANALYSIS SHOWN FURTHER DOWN IT WAS DETERMINED THAT
THE VARIABLES THAT HAVE THE BIGGEST EFFECT ARE TEMP AND HUMIDITY
this following function preprocesses that data, by only selecting these
features, fills their na values and separates them into separate cities

'''


# make function to preprocess data
def preprocess_data(data_path, labels_path=None):
    # load data and set index to city, year, weekofyear
    df = pd.read_csv(data_path)
    
    # select features we want
    '''
    features = ['ndvi_nw',
                'ndvi_se',
                'ndvi_sw',
                'precipitation_amt_mm',
                'reanalysis_air_temp_k',
                'reanalysis_avg_temp_k',
                'reanalysis_dew_point_temp_k',
                'reanalysis_max_air_temp_k',
                'reanalysis_min_air_temp_k',
                'reanalysis_precip_amt_kg_per_m2',
                'reanalysis_relative_humidity_percent',
                'reanalysis_sat_precip_amt_mm',
                'reanalysis_specific_humidity_g_per_kg',
                'reanalysis_tdtr_k',
                'station_avg_temp_c',
                'station_diur_temp_rng_c',
                'station_max_temp_c',
                'station_min_temp_c',
                'station_precip_mm']
    df = df[features]
    '''
    # fill missing values
    df.fillna(method='ffill', inplace=True)

    # add labels to dataframe
    if labels_path:
        labels = pd.read_csv(labels_path)
        #df = df.join(labels)
    
    # separate san juan and iquitos
    sj_features = df[df.city == 'sj']
    iq_features = df[df.city == 'iq']
    sj_labels = labels[labels.city == 'sj']
    iq_labels = labels[labels.city == 'iq']
    return sj_features, iq_features, sj_labels, iq_labels



In [2]:
sj_features, iq_features, sj_labels, iq_labels = preprocess_data(
                                                            'data/dengue_features_train.csv',
                                                            labels_path="data/dengue_labels_train.csv")

NameError: name 'preprocess_data' is not defined

In [1]:
type(sj_features.year)

NameError: name 'sj_features' is not defined

In [60]:
#since data is linear it makes sense to separate data linearly
#split train and test data
sj_feat_train = sj_features.head(800)
sj_labels_train = sj_labels.head(800)
sj_feat_test = sj_features.tail(sj_features.shape[0] - 800)
sj_labels_test = sj_labels.tail(sj_labels.shape[0] - 800)



iq_feat_train = iq_features.head(400)
iq_labels_train = iq_labels.head(400)
iq_feat_test = iq_features.tail(iq_features.shape[0] - 400)
iq_labels_test = iq_labels.tail(iq_labels.shape[0] - 400)

In [ ]:
iq_feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(iq_features)
sj_feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(sj_features)

In [ ]:
# Build 3 layer DNN with 10, 20, 10 units respectively
iq_regressor = tf.contrib.learn.DNNRegressor(feature_columns=iq_feature_columns, 
                                            hidden_units=[512, 256, 128], 
                                            optimizer=tf.train.AdamOptimizer(
                                                learning_rate=.03
                                            ))
sj_regressor = tf.contrib.learn.DNNRegressor(feature_columns=sj_feature_columns, 
                                            hidden_units=[512, 256, 128],
                                            optimizer=tf.train.AdamOptimizer(
                                                learning_rate=.03
                                            ))

In [ ]:
#fitting regressor iq
iq_regressor.fit(iq_feat_train, iq_labels_train.total_cases, steps=100)

In [ ]:
#fitting regressor sj
sj_regressor.fit(sj_feat_train, sj_labels_train.total_cases, steps=100)

In [ ]:
iq_predictions = list(iq_regressor.predict(iq_feat_test, as_iterable=True))
score = metrics.mean_absolute_error(iq_labels_test.total_cases, iq_predictions)
print("Mean Error: {0:f}".format(score))
#iq_labels_test.total_cases

In [ ]:
sj_predictions = list(sj_regressor.predict(sj_feat_test, as_iterable=True))
score = metrics.mean_absolute_error(sj_labels_test.total_cases, sj_predictions)
print("Mean Error: {0:f}".format(score))

In [ ]:
#NOT Great can going to add lagged variables


In [61]:
features_list = [ 'ndvi_nw',
                         'ndvi_se',
                         'ndvi_sw',
                         'precipitation_amt_mm',
                         'reanalysis_air_temp_k',
                         'reanalysis_avg_temp_k',
                         'reanalysis_dew_point_temp_k',
                         'reanalysis_max_air_temp_k',
                         'reanalysis_min_air_temp_k',
                         'reanalysis_precip_amt_kg_per_m2',
                         'reanalysis_relative_humidity_percent',
                         'reanalysis_sat_precip_amt_mm',
                         'reanalysis_specific_humidity_g_per_kg',
                         'reanalysis_tdtr_k',
                         'station_avg_temp_c',
                         'station_diur_temp_rng_c',
                         'station_max_temp_c',
                         'station_min_temp_c',
                         'station_precip_mm',]

In [62]:
#this function is from https://github.com/du-phan/DengAI/blob/master/DengAi.ipynb
def add_lagging_feature(df, lagging_feature_list, n_lag=4, remove_nan_row=False):
    
    new_df = df.copy()
    
    for original_feature in lagging_feature_list:
        for n in range(n_lag):
            lagging_feature_name = original_feature+'_lag_'+ str(n+1)
            new_df.loc[:,lagging_feature_name] = new_df.loc[:,original_feature].shift(n+1)
        
    
    #new_df = new_df.iloc[n_lag:,:]        
    
    
    for original_feature in lagging_feature_list:
        
        rolling_mean_feat_name = 'rolling_mean_' + original_feature
        rolling_std_feat_name = 'rolling_std_' + original_feature
        
        new_df.loc[:, rolling_mean_feat_name] = new_df[original_feature].rolling(window=3, center=False).mean()
        new_df.loc[:, rolling_std_feat_name] = new_df[original_feature].rolling(window=3, center=False).std()
    
    new_df.fillna(new_df.mean(), inplace=True)
    #new_df.fillna(method='ffill', inplace=True)
    return new_df

In [63]:
sj_lag = add_lagging_feature(sj_feat_train, features_list)
iq_lag = add_lagging_feature(iq_feat_train, features_list)

sj_lag_test = add_lagging_feature(sj_feat_test, features_list)
iq_lag_test = add_lagging_feature(iq_feat_test, features_list)




iq_feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(iq_lag)
sj_feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(sj_lag)
len(sj_lag)

ValueError: Data types for extracting pandas data must be int, float, or bool. Found: 'city' type='object', 'week_start_date' type='object'

In [36]:
#building DNN again
# Build 3 layer DNN with 10, 20, 10 units respectively
iq_regressor = tf.contrib.learn.DNNRegressor(feature_columns=iq_feature_columns, 
                                            hidden_units=[512, 256, 128, 64], 
                                            )
sj_regressor = tf.contrib.learn.DNNRegressor(feature_columns=sj_feature_columns, 
                                            hidden_units=[512, 256, 128, 64],
                                            )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1182b4160>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/98/8l5yvjyn5hlbh6nxjddn37jm0000gn/T/tmpic7fo8er'}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1184ddd30>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evalu

In [37]:
#fitting regressor iq
iq_regressor.fit(iq_lag, iq_labels_train.total_cases, steps=100)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Sav

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x1170f1908>, 'hidden_units': [512, 256, 128, 64], 'feature_columns': (_RealValuedColumn(column_name='', dimension=133, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x10efa2620>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [38]:
#fitting regressor sj
sj_regressor.fit(sj_lag, sj_labels_train.total_cases, steps=100)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Sav

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x1182b4550>, 'hidden_units': [512, 256, 128, 64], 'feature_columns': (_RealValuedColumn(column_name='', dimension=133, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x10efa2620>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [34]:
iq_predictions = list(iq_regressor.predict(iq_lag_test, as_iterable=True))
score = metrics.mean_absolute_error(iq_labels_test.total_cases, iq_predictions)
print("Mean Error: {0:f}".format(score))
#iq_labels_test.total_cases

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/98/8l5yvjyn5hlbh6nxjddn37jm0000gn/T/tmpnira2vrm/model.ckpt-100
Mean Error: 7.001510


In [39]:
sj_predictions = list(sj_regressor.predict(sj_lag_test, as_iterable=True))
score = metrics.mean_absolute_error(sj_labels_test.total_cases, sj_predictions)
print("Mean Error: {0:f}".format(score))
sj_predictions

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/98/8l5yvjyn5hlbh6nxjddn37jm0000gn/T/tmp_hxt4ebr/model.ckpt-100
Mean Error: 26.434311


[35.835197,
 35.972378,
 35.732738,
 36.325775,
 37.339603,
 37.960701,
 37.144135,
 37.397087,
 36.776756,
 36.456661,
 35.653423,
 35.807018,
 35.659985,
 35.486095,
 35.244411,
 35.403313,
 35.255505,
 35.689266,
 35.699947,
 35.82439,
 35.008923,
 35.561298,
 34.902836,
 34.818729,
 34.809578,
 34.717609,
 34.673023,
 34.875401,
 34.935246,
 36.598625,
 36.715099,
 36.830791,
 36.278587,
 37.313137,
 36.084736,
 36.526913,
 36.099915,
 37.072968,
 35.662216,
 36.200603,
 34.975967,
 36.325699,
 36.44923,
 36.568684,
 35.890617,
 36.518257,
 36.778053,
 36.678982,
 36.13308,
 37.299061,
 37.505581,
 36.603287,
 35.98814,
 36.100147,
 35.884331,
 35.383801,
 35.727436,
 35.961926,
 36.411839,
 35.817116,
 36.80101,
 35.955109,
 36.202469,
 35.684818,
 35.606533,
 35.055813,
 35.512527,
 35.165482,
 34.860447,
 35.373592,
 35.256874,
 35.230412,
 34.754044,
 34.953022,
 34.6161,
 34.715347,
 34.738361,
 34.817696,
 35.030678,
 35.052505,
 35.495171,
 35.474201,
 35.327938,
 36.04015,


In [ ]:
# Run model with test data -- for dengAI submission

In [ ]:
sj_features, iq_features, sj_labels, iq_labels = preprocess_data(
                                                            'data/dengue_features_test.csv',
                                                            labels_path="data/dengue_labels_test.csv")

In [ ]:
sj_predictions = list(sj_regressor.predict(sj_features, as_iterable=True))
iq_predictions = list(iq_regressor.predict(iq_features, as_iterable=True))


sj_predictions = list(map(int, sj_predictions))
iq_predictions = list(map(int, iq_predictions))

In [ ]:
sj_features, iq_features, sj_labels, iq_labels = preprocess_data(
                                                            'data/dengue_features_test.csv',
                                                            labels_path="data/dengue_labels_test.csv")

In [ ]:
test_features = pd.read_csv("data/dengue_features_test.csv",index_col=[0,1,2])
test_labels = pd.read_csv("data/dengue_labels_test.csv")
sj_labels.total_cases = sj_predictions
iq_labels.total_cases = iq_predictions
submission = pd.concat([sj_labels,iq_labels])
submission
submission.to_csv("submission/benchmark.csv", index=False)


In [ ]:
#_------------------------------------------------------------------
#EXPLANATORY ANALYSIS FOLLOWS
#-------------------------------------------------------------------

# Seperate data for San Juan
sj_train_features = train_features.loc['sj']
sj_train_labels = train_labels.loc['sj']

# Separate data for Iquitos
iq_train_features = train_features.loc['iq']
iq_train_labels = train_labels.loc['iq']

In [ ]:
print('San Juan')
print('features: ', sj_train_features.shape)
print('labels  : ', sj_train_labels.shape)

print('\nIquitos')
print('features: ', iq_train_features.shape)
print('labels  : ', iq_train_labels.shape)

In [ ]:
# Remove `week_start_date` string.
sj_train_features.drop('week_start_date', axis=1, inplace=True)
iq_train_features.drop('week_start_date', axis=1, inplace=True)

In [ ]:
# Null check
#pd.isnull(sj_train_features).any()

(sj_train_features
     .ndvi_ne
     .plot
     .line(lw=0.8))

plt.title('Vegetation Index over Time')
plt.xlabel('Time')



In [ ]:
sj_train_features.fillna(method='ffill', inplace=True)
iq_train_features.fillna(method='ffill', inplace=True)

In [ ]:
#distribution of labels
print('San Juan')
print('mean: ', sj_train_labels.mean()[0])
print('var :', sj_train_labels.var()[0])

print('\nIquitos')
print('mean: ', iq_train_labels.mean()[0])
print('var :', iq_train_labels.var()[0])

In [ ]:
sj_train_labels.hist()

In [ ]:
iq_train_labels.hist()

In [ ]:
#variance  >>  mean suggests total_cases can be described by a negative
# binomial distribution, so we'll use a negative binomial regression below.

In [ ]:
#add total cases column in features train data
sj_train_features['total_cases'] = sj_train_labels.total_cases
iq_train_features['total_cases'] = iq_train_labels.total_cases

In [ ]:
# compute correlations matrix
sj_correlations = sj_train_features.corr()
iq_correlations = iq_train_features.corr()

In [ ]:
# plot san juan
sj_corr_heat = sns.heatmap(sj_correlations)
plt.title('San Juan Variable Correlations')

In [ ]:
# plot iquitos
iq_corr_heat = sns.heatmap(iq_correlations)
plt.title('Iquitos Variable Correlations')

In [ ]:
#the total_cases variable doesn't have many obvious strong correlations.

In [ ]:
#sorted look at correlations
# San Juan
(sj_correlations
     .total_cases
     .drop('total_cases') # don't compare with myself
     .sort_values(ascending=False)
     .plot
     .barh())

In [ ]:
# Iquitos
(iq_correlations
     .total_cases
     .drop('total_cases') # don't compare with myself
     .sort_values(ascending=False)
     .plot
     .barh())

In [ ]:
#END OF BASIC EXPLORATORY ANALYSIS --------------------------------------------------------
# make function to preprocess data
def preprocess_data(data_path, labels_path=None):
    # load data and set index to city, year, weekofyear
    df = pd.read_csv(data_path, index_col=[0, 1, 2])
    
    # select features we want
    features = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c', 
                 'station_min_temp_c']
    df = df[features]
    
    # fill missing values
    df.fillna(method='ffill', inplace=True)

    # add labels to dataframe
    if labels_path:
        labels = pd.read_csv(labels_path, index_col=[0, 1, 2])
        df = df.join(labels)
    
    # separate san juan and iquitos
    sj = df.loc['sj']
    iq = df.loc['iq']
    
    return sj, iq


In [ ]:
sj_train, iq_train = preprocess_data('data/dengue_features_train.csv',
                                    labels_path="data/dengue_labels_train.csv")

In [ ]:
sj_train.describe()

In [ ]:
iq_train.describe()